# Introduction
This notebook is used to combine the cleaned versions of all datasets used in this project into one joined dataframe. This data will then be exported as a csv to be used in the main project.

# Datasets
The following datasets will be merged
- Weather Dataset
- MPD (Milwaukee Police Department) Dataset

# Imports
These are the libraries that will be relvant for cleaning this dataset.

In [17]:
import pandas as pd
import numpy as np

# Cleaning the Dataset
The following sections walk through the steps used to combine the weather dataset into the MPD dataset.

## Load the Raw Data
This section loads the raw data and examines how it is originally formatted.

### Load the MPD Data

In [18]:
mpd_data = pd.read_csv("mpd_data_cleaned.csv")

In [19]:
mpd_data.head()

,district,nature,status,call_id,datetime,traffic_crime,weapon_crime,isCorner,houseNumber,primaryStreetName,primaryStreetSuffix,secondaryStreetName,secondaryStreetSuffix
0,4,ACC PI,Service in Progress,191411633,2019-05-21 15:19:03,False,False,False,7420.0,W GOOD HOPE,RD,NaN,NaN
1,3,TRAFFIC STOP,City Citation(s) Issued,191411672,2019-05-21 15:24:30,True,False,False,1421.0,N 27TH,ST,NaN,NaN
2,7,SUBJ WANTED,Assignment Completed,191411674,2019-05-21 15:25:46,False,False,False,4054.0,N 71ST,ST,NaN,NaN
3,2,SPECIAL ASSIGN,Service in Progress,191412545,2019-05-21 20:46:28,False,False,False,245.0,W LINCOLN,AV,NaN,NaN
4,3,TRBL W/SUBJ,Unable to Locate Complainant,191412465,2019-05-21 20:50:03,False,False,False,1721.0,W CANAL,ST,NaN,NaN


### Load the Weather Data

In [20]:
weather_data = pd.read_csv("weather_data_cleaned.csv")

In [21]:
weather_data.head()

,loc_id,isdaytime,tempC,windspeedKmph,winddirdegree,weatherDesc,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,datetime
0,1,False,14,11,241,Clear,0.0,77,10,1016,3,14,11,14,23,14,1,2008-07-01 00:00:00
1,1,False,14,12,246,Clear,0.0,77,10,1016,10,14,10,14,25,14,1,2008-07-01 01:00:00
2,1,False,14,12,251,Clear,0.0,76,10,1015,17,14,10,14,26,14,1,2008-07-01 02:00:00
3,1,False,15,13,256,Clear,0.0,76,10,1015,24,15,10,14,27,14,1,2008-07-01 03:00:00
4,1,False,15,12,255,Clear,0.0,74,10,1016,22,15,10,14,26,14,1,2008-07-01 04:00:00


## Fix Feature Formats
The features are likely in the wrong formats based on what they have just been loaded as. They must be changed to their correct types in order for the merge to work.

### Fixing Feature Formats in the MPD Dataset

In [22]:
mpd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   district               object 
 1   nature                 object 
 2   status                 object 
 3   call_id                int64  
 4   datetime               object 
 5   traffic_crime          bool   
 6   weapon_crime           bool   
 7   isCorner               bool   
 8   houseNumber            float64
 9   primaryStreetName      object 
 10  primaryStreetSuffix    object 
 11  secondaryStreetName    object 
 12  secondaryStreetSuffix  object 
dtypes: bool(3), float64(1), int64(1), object(8)
memory usage: 318.8+ MB


Many of the features in the MPD dataset are improperly formatted. The district, nature, status, primaryStreetName, primaryStreetSuffix, secondaryStreetName, and secondaryStreetSuffix all need to become categorical features. The datetime feature needs to be changed to datetime.

In [23]:
mpd_data['district'] = mpd_data['district'].astype('category')
mpd_data['nature'] = mpd_data['nature'].astype('category')
mpd_data['status'] = mpd_data['status'].astype('category')
mpd_data['primaryStreetName'] = mpd_data['primaryStreetName'].astype('category')
mpd_data['primaryStreetSuffix'] = mpd_data['primaryStreetSuffix'].astype('category')
mpd_data['secondaryStreetName'] = mpd_data['secondaryStreetName'].astype('category')
mpd_data['secondaryStreetSuffix'] = mpd_data['secondaryStreetSuffix'].astype('category')
mpd_data['datetime'] = pd.to_datetime(mpd_data['datetime'], infer_datetime_format=True)

### Fixing Feature Formats in the MPD Dataset

In [24]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119112 entries, 0 to 119111
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   loc_id         119112 non-null  int64  
 1   isdaytime      119112 non-null  bool   
 2   tempC          119112 non-null  int64  
 3   windspeedKmph  119112 non-null  int64  
 4   winddirdegree  119112 non-null  int64  
 5   weatherDesc    119112 non-null  object 
 6   precipMM       119112 non-null  float64
 7   humidity       119112 non-null  int64  
 8   visibilityKm   119112 non-null  int64  
 9   pressureMB     119112 non-null  int64  
 10  cloudcover     119112 non-null  int64  
 11  HeatIndexC     119112 non-null  int64  
 12  DewPointC      119112 non-null  int64  
 13  WindChillC     119112 non-null  int64  
 14  WindGustKmph   119112 non-null  int64  
 15  FeelsLikeC     119112 non-null  int64  
 16  uvIndex        119112 non-null  int64  
 17  datetime       119112 non-nul

Everything looks good with the weather data except for datetime and weatherDesc. WeatherDesc needs to become a categorical feature and datetime needs to become a datetime feature.

In [25]:
weather_data['weatherDesc'] = weather_data['weatherDesc'].astype('category')
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], infer_datetime_format=True)

## Sort the Datasets by Time
The datasets must be [sorted](https://www.geeksforgeeks.org/how-to-sort-a-pandas-dataframe-by-date/) in an increasing manner based on the target feature that is used to join the two datasets. The datetime feature will be used to sort because the goal is to include the weather at the time a crime occured.

### Sort the MPD Data

In [26]:
mpd_data = mpd_data.sort_values(by='datetime')

### Sort the Weather Data

In [27]:
weather_data = weather_data.sort_values(by='datetime')

## Join the Dataframes
The pandas [merge_asof](https://pandas.pydata.org/docs/dev/reference/api/pandas.merge_asof.html) method can be used to quickly [merge the two sorted datasets](https://www.geeksforgeeks.org/merge-two-pandas-dataframes-based-on-closest-datetime/) into one on the closest time. The merge will be conducted so that the nearest hour that matches between the two datasets will be used. This should lead to df maintaining the same number of rows but adding on the number of columns in weather_data.

In [28]:
print("MPD Data Shape Before: %s" % ((mpd_data.shape), ))
print("Weather Data Shape Before: %s" % ((weather_data.shape), ))
merged_data = pd.merge_asof(mpd_data, weather_data, on='datetime', direction='nearest')  
print("Merged Data Shape After: %s" % ((merged_data.shape), ))

MPD Data Shape Before: (4027695, 13)
Weather Data Shape Before: (119112, 18)
Merged Data Shape After: (4027695, 30)


# Conclusion
At this point the datasets are done being merged. The last steps are to show the final outputs of the [`.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), [`.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html), and [`.describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) methods and output the merged data to a new csv file.

In [29]:
merged_data.head()

,district,nature,status,call_id,datetime,traffic_crime,weapon_crime,isCorner,houseNumber,primaryStreetName,...,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
0,NaN,FOLLOW UP,Assignment Completed,163120850,2016-11-07 09:33:52,False,False,False,2920.0,N 4TH,...,64,10,1026,1,14,7,14,18,14,4
1,NaN,PROPERTY DAMAGE,Advised/Referral,163120774,2016-11-07 09:33:55,False,False,False,5929.0,W BLUE MOUND,...,64,10,1026,1,14,7,14,18,14,4
2,NaN,VACANT HOUSE CHK,Assignment Completed,163120851,2016-11-07 09:34:58,False,False,False,3425.0,N 93RD,...,64,10,1026,1,14,7,14,18,14,4
3,NaN,FOLLOW UP,Assignment Completed,163120853,2016-11-07 09:36:20,False,False,False,245.0,W LINCOLN,...,64,10,1026,1,14,7,14,18,14,4
4,NaN,TRAFFIC STOP,Advised,163120857,2016-11-07 09:38:10,True,False,False,765.0,N 25TH,...,64,10,1026,1,14,7,14,18,14,4


In [30]:
merged_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4027695 entries, 0 to 4027694
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   district               3937463 non-null  category      
 1   nature                 4027695 non-null  category      
 2   status                 4027695 non-null  category      
 3   call_id                4027695 non-null  int64         
 4   datetime               4027695 non-null  datetime64[ns]
 5   traffic_crime          4027695 non-null  bool          
 6   weapon_crime           4027695 non-null  bool          
 7   isCorner               4027695 non-null  bool          
 8   houseNumber            3426083 non-null  float64       
 9   primaryStreetName      4016778 non-null  category      
 10  primaryStreetSuffix    3844881 non-null  category      
 11  secondaryStreetName    549421 non-null   category      
 12  secondaryStreetSuffix  53185

In [31]:
merged_data.describe()

,call_id,houseNumber,loc_id,tempC,windspeedKmph,winddirdegree,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
count,4.027695e+06,3.426083e+06,4027695.0,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06
mean,1.891808e+08,3.367593e+03,1.0,9.332960e+00,1.396831e+01,1.978097e+02,8.068526e-02,7.252384e+01,9.280653e+00,1.016711e+03,4.627480e+01,9.741805e+00,4.295812e+00,7.040703e+00,2.030895e+01,7.329460e+00,2.161411e+00
std,1.459309e+07,2.415387e+03,0.0,1.099667e+01,6.506821e+00,9.301290e+01,3.459743e-01,1.560838e+01,1.861185e+00,7.728984e+00,3.620323e+01,1.150243e+01,1.050146e+01,1.314242e+01,9.506874e+00,1.356433e+01,1.869588e+00
min,1.630815e+08,1.000000e+00,1.0,-3.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,9.860000e+02,0.000000e+00,-3.000000e+01,-3.400000e+01,-4.400000e+01,0.000000e+00,-4.400000e+01,1.000000e+00
25%,1.732129e+08,1.614000e+03,1.0,1.000000e+00,9.000000e+00,1.260000e+02,0.000000e+00,6.100000e+01,1.000000e+01,1.012000e+03,1.100000e+01,1.000000e+00,-4.000000e+00,-3.000000e+00,1.300000e+01,-3.000000e+00,1.000000e+00
50%,1.906707e+08,2.920000e+03,1.0,9.000000e+00,1.300000e+01,2.140000e+02,0.000000e+00,7.400000e+01,1.000000e+01,1.016000e+03,3.900000e+01,9.000000e+00,4.000000e+00,6.000000e+00,1.900000e+01,6.000000e+00,1.000000e+00
75%,2.016115e+08,4.600000e+03,1.0,1.900000e+01,1.800000e+01,2.760000e+02,0.000000e+00,8.500000e+01,1.000000e+01,1.021000e+03,8.300000e+01,1.900000e+01,1.400000e+01,1.900000e+01,2.600000e+01,1.900000e+01,3.000000e+00
max,2.201010e+08,6.460500e+05,1.0,3.500000e+01,4.700000e+01,3.600000e+02,1.630000e+01,1.000000e+02,1.000000e+01,1.048000e+03,1.000000e+02,4.100000e+01,2.700000e+01,3.500000e+01,7.100000e+01,4.100000e+01,9.000000e+00


In [32]:
merged_data.to_csv("merged_data.csv", index=False)